## <a href='https://projecteuler.net/problem=31'>31. Coin sums</a>
In England the currency is made up of pound, £, and pence, p, and there are eight coins in general circulation:

1p, 2p, 5p, 10p, 20p, 50p, £1 (100p) and £2 (200p).

It is possible to make £2 in the following way:

1×£1 + 1×50p + 2×20p + 1×5p + 1×2p + 3×1p

How many different ways can £2 be made using any number of coins?
___

idea from <a href='https://www.youtube.com/watch?v=VLbePGBOVeg'>youtube</a>:  
for how many ways a coin set of {1p, 2p, 5p, 10p, 20p, 50p, 100p, 200p} to make 200p,  
is the same as asking how many ways of a 200p coin change using a coin set of {1p, 2p, 5p, 10p, 20p, 50p, 100p, 200p}.  

Generally, it is:  

let:  
$$ 
p(x) = (1+x^1+x^2+...+x^{200}) \cdot (1+x^2+x^4+...+x^{200})  \cdot (1+x^5+x^{10}+...+x^{200})  \cdot (1+x^{10}+x^{20}+...+x^{200})  \cdot (1+x^{20}+x^{40}+...+x^{200})  \cdot (1+x^{50}+x^{100}+x^{150}+x^{200})  \cdot (1+x^{100}+x^{200})  \cdot (1+x^{200}) 
$$
the coefficiect of $x^{200}$ of $p(x)$ is the answer.  

this idea can be extended to do integer partition, simply speaking:  
how many ways of an integer k coin change using a coin set of {1 unit, 2 units, 3 units, 4 units, ... ,k units} 
$\equiv$ integer partition

___
for this question, a function to multiply polynomial and obtain the coefficients is required,  
then use dictionery for storage

In [1]:
def polynomial_product(coef1: list, coef2: list) -> list: 
    
    new_coef_get = {}
    
    for power1, c1 in enumerate(coef1):
        for power2, c2 in enumerate(coef2):
            new_coef_get[power1+power2] = new_coef_get.get(power1+power2, 0) + c1*c2
            
    # to list
    new_coef = []
    for power in new_coef_get:
        new_coef.append(new_coef_get[power])
            
    return new_coef

In [2]:
# input 
q31_input = {'pence_set': [1, 2, 5, 10, 20, 50, 100, 200], 'target': 200}

# function 
def q31(pence_set: list, target: list): 
    
    # store all required polynomial
    polynomial_set = dict()
    for pence in pence_set: 
        coef_poly = [1 if i%pence == 0 else 0 for i in range(0, target+1)]
        polynomial_set[pence] = polynomial_set.get(pence, coef_poly)
        
    # multiply polynomial
    new_polynomial = [1]
    for polynomial in polynomial_set:
        new_polynomial = polynomial_product(new_polynomial, polynomial_set[polynomial])
        
    return print(f'there are {new_polynomial[target]} ways to coin change {target} pence with {pence_set} pence coins')

In [3]:
%%timeit -n 1 -r 1
q31(**q31_input)

there are 73682 ways to coin change 200 pence with [1, 2, 5, 10, 20, 50, 100, 200] pence coins
284 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


___
recall back to the integer partition,  
let see how well the function above does

In [4]:
# modify function 
def integer_partition(n: int) -> int: 
    
    # required list
    below_int = [j for j in range(1, n+1)]
    
    # store all required polynomial
    polynomial_set = dict()
    for i in below_int: 
        coef_poly = [1 if j%i == 0 else 0 for j in range(0, n+1)]
        polynomial_set[i] = polynomial_set.get(i, coef_poly)
        
    # multiply polynomial
    new_polynomial = [1]
    for polynomial in polynomial_set:
        new_polynomial = polynomial_product(new_polynomial, polynomial_set[polynomial])
        
    return new_polynomial[n]

for i in range(20): 
    integer_partition_input = {'n': i}
    print(integer_partition(**integer_partition_input), end=', ')

1, 1, 2, 3, 5, 7, 11, 15, 22, 30, 42, 56, 77, 101, 135, 176, 231, 297, 385, 490, 

seems like it works, but obviously the code is kinda inefficient, so be careful when using it

___
old codes below

In [5]:
201 * 101 * 41 * 21 * 11 * 5 * 3 * 2

5768123130

in total, if using brute force, there will be 5768123130 combinations to check, gna take forever

so, consider a simpler case:  
in how many way, that 1p and 2p can be arranged in to the multiples of 5p?

In [6]:
import numpy as np
import math
import time

cnt = 0

for i in range(200 + 1):
    
    for j in range(100 + 1):
        
        money = i * 1 + j * 2
        
        if money%5 == 0 and money <= 200:  # the mod5 is because 5p is excluded
            cnt += 1
            
cnt

2081

___
to get 200p:
1. 450 ways to get 50p with only 1p,2p,5p,10p,20p <br>
will bind with 3(50p) or 1(1£)+1(50p) --> twice as much of this case
2. 4111 ways to get 100p with only 1p,2p,5p,10p,20p <br>
will bind with 2(50p) or 1(1£) --> twice as much of this case
3. 16860 ways to get 150p with only 1p,2p,5p,10p,20p <br>
will bind with only 1(50p)
4. 16860 ways to get 200p with only 1p,2p,5p,10p,20p <br>
will bind with no more coins <br><br>
also rmb cases without any 1p,2p,5p,10p,20p: <br><br>
1. 1(2£)
2. 2(1£)
3. 4(50p)
4. 1(1£)+2(50p)

In [7]:
'''
# multiples of 50p

# brute force
t1 = time.time()

# cnt = [50,100,150,200]
cnt = np.zeros(4)

for i in range(200 + 1):
    
    for j in range(100 + 1):
        
        for k in range(40+1):
            
            for ii in range(20+1):
                
                for jj in range(10+1):
                               
                    money = i * 1 + j * 2 + k * 5 + ii * 10 + jj * 20
        
                    if money == 50:
                        cnt[0] += 1
                    
                    if money == 100:
                        cnt[1] += 1
                        
                    if money == 150:
                        cnt[2] += 1
                        
                    if money == 200:
                        cnt[3] += 1
            
t2 = time.time()
print(t2-t1,'seconds taken')

cnt
'''
print('output:', '\n73.02263712882996 seconds taken\narray([  450.,  4111., 16860., 47696.])')

output: 
73.02263712882996 seconds taken
array([  450.,  4111., 16860., 47696.])


___
some faster code: 

In [8]:
# eg, from https://projecteuler.net/thread=31;page=9, veronica248yu
count=1

for x1 in range(3):
    for x2 in range(1+int((200-x1*100)/50)):
        for x3 in range(1+int((200-x1*100-x2*50)/20)):
            for x4 in range(1+int((200-x1*100-x2*50-x3*20)/10)):
                for x5 in range(1+int((200-x1*100-x2*50-x3*20-x4*10)/5)):
                    for x6 in range(1+int((200-x1*100-x2*50-x3*20-x4*10-x5*5)/2)):
                        #You can also calculate x7 if you want, but it is not necessary
                        count+=1

print('Number of combinations:',count)

Number of combinations: 73682
